In [1]:
import sys
import glob,os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
sys.path.append("../")
import utils.data_utils as du
import utils.visualization_utils as vu

In [2]:
# Init paths
path_raw = os.environ.get("DES_DATA")
path_dump = './dump/data_skimming/'
path_bazin = '../dumps/bazin_SNNone/'
debug=False

In [3]:
# load complete data
# do run: process_Data_for_notebooks.py first! the notebook crashes with such large data to process
df_real = du.load_all_data('dummy','../dumps/preprocessed/real/', redo_photometry =False)
df_fake = du.load_all_data('dummy','../dumps/preprocessed/fake/', redo_photometry =False)

[Loading preprocessed data ../dumps/preprocessed/real/ ] 
[Loading preprocessed data ../dumps/preprocessed/fake/ ] 


In [ ]:
df_group_fake = df_fake.groupby("SNID").mean()

In [ ]:
len(df_group_fake)

In [ ]:
#sanity check for bazin
# only for fakes
df = df_group_fake
fig = plt.figure()
df["PEAKMJDNORM"] = df['PRIVATE(DES_fake_peakmjd)']
plt.hist(df["PEAKMJDNORM"]-df["PKMJDINI"],histtype="step",label='bazin')
df = df[df["PEAKMJDNORM"]-df["PRIVATE(DES_mjd_trigger)"]<1500]
plt.hist(df["PEAKMJDNORM"]-df["PRIVATE(DES_mjd_trigger)"],histtype="step",label='trigger')
plt.yscale("log")
plt.legend()

In [ ]:
# see why some SNe are missclassified
# plot some lcs

In [ ]:
# load skimmed data
# df_real_bazin = du.load_all_data(f'{path_bazin}/real/','../dumps/bazin_SNNone/real/pickle/', redo_photometry =True)
df_fake_bazin = du.load_all_data(f'{path_bazin}/fake/','../dumps/bazin_SNNone/fake/pickle/', redo_photometry = True)

In [ ]:
# load SNN predictions
name_model = "vanilla_S_0_CLF_2_R_None_photometry_DF_1.0_N_global_lstm_32x2_0.05_128_True_mean_C"
file_pred = "PRED_vanilla_S_0_CLF_2_R_None_photometry_DF_1.0_N_global_lstm_32x2_0.05_128_True_mean_C.pickle"
preds = pd.read_pickle(f"bazin_SNNone")

In [ ]:
# plot P vs. E, P vs. z, E vs. z